In [4]:
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
import spacy
import time

nlp = spacy.load('en_core_web_sm')
stemmer = PorterStemmer()
stop_words = nlp.Defaults.stop_words


def preprocess_text(text):
    doc = nlp(text)
    preprocessed_tokens = [
        token.lemma_ for token in doc if token.text.lower() not in stop_words
        ]
    return " ".join([stemmer.stem(token) for token in preprocessed_tokens])


2023-10-23 18:02:51.087498: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import pandas as pd
import numpy as np
import string
# FRE SCORE
from textstat import flesch_reading_ease
# SENTIMENT ANALYSIS USING VADER
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#POS TAGGING
import spacy
#VECTORISING TEXT AND CREATING PIPELINE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
#COSINE SIMILARITY BETWEEN REVIEWS
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
df = pd.read_csv("WELFake_Dataset.csv")

In [7]:
df.columns

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')

In [8]:
#RENAME COLUMNS
df.rename(columns={'text': 'NEWS TEXT'}, inplace=True)
df.rename(columns={'title': 'NEWS TITLE'}, inplace=True)
df.rename(columns={'label': 'LABEL'}, inplace=True)


In [9]:
# df['PREPROCESSED TEXT'] = df['NEWS TEXT'].apply(preprocess_text)

In [10]:
def add_readability_score(df):
    df['READABILITY_FRE'] = df['NEWS TEXT'].apply(
        lambda d: flesch_reading_ease(d))

def add_title_length(df):
    df['TITLE_LENGTH'] = df['NEWS TITLE'].apply(lambda d: len(d))

def add_text_length(df):
    df['TEXT LENGTH'] = df['NEWS TEXT'].apply(lambda d: len(d))

def add_vader_text_sentiment_score(df):
    sid = SentimentIntensityAnalyzer()

    df['TEXT SENTIMENT SCORE'] = df['NEWS TEXT'].apply(
        lambda d: sid.polarity_scores(d)['compound'])
    
def add_vader_title_sentiment_score(df):
    sid = SentimentIntensityAnalyzer()

    df['TITLE SENTIMENT SCORE'] = df['NEWS TITLE'].apply(
        lambda d: sid.polarity_scores(d)['compound'])
    
def add_sentiment_category(df, threshold):

    def assign_sentiment_category(score):
        if score > threshold:
            return 1
        else:
            return 0

    df['SENTIMENT CATEGORY'] = df['TEXT SENTIMENT SCORE'].apply(
        assign_sentiment_category)
    
def add_count_punctuation(df):
    def count_punctuation(text):
        punctuation_count = sum([1 for char in text if char in string.punctuation])
        return punctuation_count
    
    # Assuming you have a DataFrame called 'df' with a column 'REVIEW_TEXT'
    df['TEXT PUNCTUATION COUNT'] = df['NEWS TEXT'].apply(count_punctuation)

def add_count_capital_chars(df):
    def count_capital_chars(text):
        capital_count = sum([1 for char in text if char.isupper()])
        return capital_count


    # Assuming you have a DataFrame called 'df' with a column 'REVIEW_TEXT'
    df['TEXT CAPITAL CHARS'] = df['NEWS TEXT'].apply(count_capital_chars)

def add_pos_tags(df):
    def count_pos(Pos_counts, pos_type):
        pos_count = Pos_counts.get(pos_type, 0)
        return pos_count

    def pos_counts(text):
        doc = nlp(text)
        Pos_counts = doc.count_by(spacy.attrs.POS)
        return Pos_counts

    poscounts =  df["NEWS TEXT"].apply(pos_counts)
    df['TEXT NUM NOUNS'] = df['NEWS TEXT'].apply(
        lambda text: count_pos(poscounts, spacy.parts_of_speech.NOUN))
    df['TEXT NUM VERBS'] = df['NEWS TEXT'].apply(
        lambda text: count_pos(poscounts, spacy.parts_of_speech.VERB))
    df['TEXT NUM ADJECTIVES'] = df['NEWS TEXT'].apply(
        lambda text: count_pos(poscounts, spacy.parts_of_speech.ADJ))
    df['TEXT NUM ADVERBS'] = df['NEWS TEXT'].apply(
        lambda text: count_pos(poscounts, spacy.parts_of_speech.ADV))
    
def add_named_entities(df):
    def count_entities(text):
        doc = nlp(text)
        ent_count = len([ent.text for ent in doc.ents])
        return ent_count

    df['TEXT NUM NAMED ENTITIES'] = df['NEWS TEXT'].apply(count_entities)
    
def calculate_lexical_diversity(text):
    words = text.split()
    total_words = len(words)
    unique_words = len(set(words))
    
    if total_words > 0:
        lexical_diversity = unique_words / total_words
    else:
        lexical_diversity = 0.0
    
    return lexical_diversity

In [11]:
df.isnull().sum()

Unnamed: 0      0
NEWS TITLE    558
NEWS TEXT      39
LABEL           0
dtype: int64

In [12]:
#REMOVE NULL RECORDS
df = df.dropna()

In [13]:
#USE ONLY 20,000 RECORDS
df1 = df[:20000]

In [14]:
#ADD COLUMN FOR NUMBER OF WORDS IN TEXT
df1['TEXT WORD COUNT'] = df1['NEWS TEXT'].apply(lambda x: len(x.split()))

/var/folders/tl/ps_r0r591fx137cf06vhn3tm0000gn/T/ipykernel_65632/88854239.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['TEXT WORD COUNT'] = df1['NEWS TEXT'].apply(lambda x: len(x.split()))


In [15]:
#ADD COLUMN FOR NUMBER OF WORDS IN TITLE
df1["TITLE WORD COUNT"] = df1["NEWS TITLE"].apply(lambda x: len(x.split()))

/var/folders/tl/ps_r0r591fx137cf06vhn3tm0000gn/T/ipykernel_65632/1390510365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["TITLE WORD COUNT"] = df1["NEWS TITLE"].apply(lambda x: len(x.split()))


In [16]:
#ADD TEXT LENGTH AND TITLE LENGTH
add_text_length(df1)
add_title_length(df1)

/var/folders/tl/ps_r0r591fx137cf06vhn3tm0000gn/T/ipykernel_65632/3849230110.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TEXT LENGTH'] = df['NEWS TEXT'].apply(lambda d: len(d))
/var/folders/tl/ps_r0r591fx137cf06vhn3tm0000gn/T/ipykernel_65632/3849230110.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TITLE_LENGTH'] = df['NEWS TITLE'].apply(lambda d: len(d))


In [17]:
#ADD SENTIMENT SCORE OF TEXT
add_vader_text_sentiment_score(df1)

/var/folders/tl/ps_r0r591fx137cf06vhn3tm0000gn/T/ipykernel_65632/3849230110.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TEXT SENTIMENT SCORE'] = df['NEWS TEXT'].apply(


In [18]:
df1.columns

Index(['Unnamed: 0', 'NEWS TITLE', 'NEWS TEXT', 'LABEL', 'TEXT WORD COUNT',
       'TITLE WORD COUNT', 'TEXT LENGTH', 'TITLE_LENGTH',
       'TEXT SENTIMENT SCORE'],
      dtype='object')

In [19]:
#ADD VADER SENTIMENT SCORE OF TITLE
add_vader_title_sentiment_score(df1)

/var/folders/tl/ps_r0r591fx137cf06vhn3tm0000gn/T/ipykernel_65632/3849230110.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TITLE SENTIMENT SCORE'] = df['NEWS TITLE'].apply(


In [20]:
#ADD READABILITY SCORE OF TEXT
add_readability_score(df1)

/var/folders/tl/ps_r0r591fx137cf06vhn3tm0000gn/T/ipykernel_65632/3849230110.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['READABILITY_FRE'] = df['NEWS TEXT'].apply(


In [21]:
#ADD CAPITAL CHARACTER COUNT
add_count_capital_chars(df1)

/var/folders/tl/ps_r0r591fx137cf06vhn3tm0000gn/T/ipykernel_65632/3849230110.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TEXT CAPITAL CHARS'] = df['NEWS TEXT'].apply(count_capital_chars)


In [22]:
#ADD PUNCTUATION COUNT
add_count_punctuation(df1)

/var/folders/tl/ps_r0r591fx137cf06vhn3tm0000gn/T/ipykernel_65632/3849230110.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TEXT PUNCTUATION COUNT'] = df['NEWS TEXT'].apply(count_punctuation)


In [23]:
def add_pos_tags(df):
    def count_pos(Pos_counts, pos_type):
        pos_count = Pos_counts.get(pos_type, 0)
        return pos_count

    def pos_counts(text):
        doc = nlp(text)
        Pos_counts = doc.count_by(spacy.attrs.POS)
        return Pos_counts

    poscounts =  df["NEWS TEXT"].apply(pos_counts)
    df['TEXT NUM NOUNS'] = df['NEWS TEXT'].apply(
        lambda text: count_pos(poscounts, spacy.parts_of_speech.NOUN))
    df['TEXT NUM VERBS'] = df['NEWS TEXT'].apply(
        lambda text: count_pos(poscounts, spacy.parts_of_speech.VERB))
    df['TEXT NUM ADJECTIVES'] = df['NEWS TEXT'].apply(
        lambda text: count_pos(poscounts, spacy.parts_of_speech.ADJ))
    df['TEXT NUM ADVERBS'] = df['NEWS TEXT'].apply(
        lambda text: count_pos(poscounts, spacy.parts_of_speech.ADV))

In [24]:
#ADD POS TAGS
add_pos_tags(df1)

KeyboardInterrupt: 

In [ ]:
df2 = df1[:10000]

In [ ]:
#ADD COUNT OF NAMES ENTITIES IN TEXT
add_named_entities(df2)

In [ ]:
#ADD SENTIMENT CATEGORY OF TEXT
add_sentiment_category(df1)

In [27]:
#SAVE AS CSV FILE

df1.to_csv("training_data_set_1.csv")

In [26]:
#ADD LEXICAL DIVERSITY
df['TEXT LEXICAL DIVERSITY'] = df['NEWS TEXT'].apply(calculate_lexical_diversity)

In [28]:
#TRAIN MODELS
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.linear_model import RidgeClassifier, SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
classifiers = {}
classifiers.update({"XGBClassifier": XGBClassifier(eval_metric='logloss',objective='binary:logistic',use_label_encoder=False)})
# classifiers.update({"CatBoostClassifier": CatBoostClassifier(silent=True)})
classifiers.update({"LinearSVC": LinearSVC(max_iter=10000)})
# classifiers.update({"MultinomialNB": MultinomialNB()})
# classifiers.update({"LGBMClassifier": LGBMClassifier()})
classifiers.update({"RandomForestClassifier": RandomForestClassifier()})
classifiers.update({"DecisionTreeClassifier": DecisionTreeClassifier()})
classifiers.update({"ExtraTreeClassifier": ExtraTreeClassifier()})
classifiers.update({"AdaBoostClassifier": AdaBoostClassifier()})
classifiers.update({"KNeighborsClassifier": KNeighborsClassifier()})
classifiers.update({"RidgeClassifier": RidgeClassifier()})
classifiers.update({"SGDClassifier": SGDClassifier()})
classifiers.update({"BaggingClassifier": BaggingClassifier()})
classifiers.update({"BernoulliNB": BernoulliNB()})
classifiers.update({"LogisticRegression": LogisticRegression()})
classifiers.update({"SVM": SVC()})

/Users/kpandey/anaconda3/lib/python3.10/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [ ]:
df1.columns

In [30]:
features_numeric = [
    'TEXT WORD COUNT', 'TITLE WORD COUNT', 'TEXT LENGTH', 'TITLE_LENGTH',
       'TEXT SENTIMENT SCORE', 'TITLE SENTIMENT SCORE', 'READABILITY_FRE',
       'TEXT CAPITAL CHARS', 'TEXT PUNCTUATION COUNT']

In [31]:
X = df1[features_numeric]
Y = df1["LABEL"]

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)

In [33]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [34]:
# CREATE A DATAFRAME OF MODELS WITH RUN TIME AND AUC SCORES
df_models = pd.DataFrame(
    columns=['model', 'run_time', 'accuracy', 'precision', 'f1_score'])

for key in classifiers:
    # STARTING TIME
    start_time = time.time()
    # CURRENT CLASSIFIER
    clf = classifiers[key]
    #TRAIN CLASSIFIER ON TRAINING DATA
    clf.fit(X_train_scaled, y_train)
    # MAKE PREDICTIONS USING CURRENT CLASSIFIER
    predictions = clf.predict(X_test_scaled)
    # CALCULATE ACCURACY
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    f1score = f1_score(y_test, predictions)
    
    row = {'model': key,
           'run_time': format(round((time.time() - start_time)/60, 2)),
           'accuracy': accuracy,
           'precision': precision,
           'f1_score': f1score
           }

    df_models = df_models._append(row, ignore_index=True)

df_models = df_models.sort_values(by='accuracy', ascending=False)

# PRINT THE MODELS WITH AUC SCORES [SORTED]
print(df_models)

/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


                     model run_time  accuracy  precision  f1_score
0            XGBClassifier     0.01   0.80050   0.823795  0.804412
2   RandomForestClassifier     0.07   0.79400   0.813181  0.799318
9        BaggingClassifier     0.02   0.78475   0.820700  0.784804
12                     SVM     0.25   0.77275   0.822660  0.767816
5       AdaBoostClassifier     0.01   0.74825   0.787042  0.746411
6     KNeighborsClassifier     0.01   0.74500   0.791485  0.739796
3   DecisionTreeClassifier      0.0   0.71875   0.742081  0.724062
8            SGDClassifier      0.0   0.69900   0.774534  0.674419
11      LogisticRegression      0.0   0.69525   0.722024  0.698640
4      ExtraTreeClassifier      0.0   0.69350   0.712945  0.701848
7          RidgeClassifier      0.0   0.69175   0.722772  0.692288
1                LinearSVC     0.19   0.69125   0.720413  0.693015
10             BernoulliNB      0.0   0.66425   0.700377  0.659742


In [35]:
import pickle

dictionary = {}

dictionary["MODEL"] = classifiers["XGBClassifier"]

dictionary["SCALER"] = scaler

with open("model_1.pkl", "wb") as file:
    pickle.dump(dictionary, file)